In [1]:
import os
os.chdir('../..')
from pipelines.util import *
import pandas as pd
from datetime import datetime
from collections import Counter
import re

In [2]:
data = pd.read_csv(os.path.join(WDIR, 'true-north/true_north_clean.csv'))
data.drop(columns='associated Company Domain (Contact Level)', inplace=True)
data[data['Industry'].isna()]

,last_updated,Create Date,Do you feel the True North report identified the key challenges and opportunities facing the region?,Are you interested in attending future True North events?,Are you currently a B Corp or in the process of becoming a B Corp?,Would you be interested in hearing more from Brabners about the B Corp process?,Company Name,City,Industry,sector,location,Which theme of the True North report do you most identify with and could support activity around?,How would you like to be involved with the True North network?,company_size
2,2024-07-12 10:00:00,2023-05-01 00:00:00,Yes,Yes,No,Yes,Freshfield PR Ltd,NaN,NaN,Marketing,NaN,"People, skills and the future",NaN,NaN
8,2024-07-10 09:00:00,2024-03-11 11:09:00,Yes,Yes,No,No,Marsden Strategy,NaN,NaN,Public Affairs Consultancy,Greater Manchester,"People, skills and the future; Innovation and ...",Keeping informed about the latest True North n...,0-9
12,2024-07-08 18:01:00,2024-03-07 07:09:00,Yes,Yes,No,No,Sustainable Ventures,Greater Manchester,NaN,Business admin and support,Greater Manchester,Innovation and change; Purpose and societal im...,Keeping informed about the latest True North n...,0-9
18,2024-07-01 08:06:00,2024-02-06 13:56:00,Yes,Yes,No,Yes,Essential Sourcing,Blackburn,NaN,Procurement & Sourcing,Lancashire,"People, skills and the future; Innovation and ...",Keeping informed about the latest True North n...,0-9
26,2024-06-25 17:18:00,2024-06-25 17:18:00,Yes,Yes,Yes,Yes,Pet Trust UK,NaN,NaN,Technology and Software Development; Other - p...,Greater Manchester,"People, skills and the future; Innovation and ...",Keeping informed about the latest True North n...,0-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,2023-05-01 00:00:00,2023-05-01 00:00:00,Yes,Yes,No,No,Hatch Recruitment Ltd,Eccles,NaN,Recruitment,Lancashire; West Yorkshire; Greater Manchester...,"People, skills and the future",Keeping informed about the latest True North n...,0-9
266,2023-05-01 00:00:00,2023-05-01 00:00:00,Yes,Yes,No,No,Open Awards,Speke,NaN,Education,Merseyside,"People, skills and the future; Innovation and ...",Keeping informed about the latest True North n...,10-49
267,2023-05-01 00:00:00,2023-05-01 00:00:00,Yes,Yes,Yes,Yes,Big Hand 4 Business,NaN,NaN,Accountancy,NaN,Innovation and change,NaN,NaN
270,2023-04-26 11:30:00,2023-08-10 14:38:00,Yes,Yes,No,Yes,Healthcare world,NaN,NaN,"Health, digital, innovation",NaN,Innovation and change,NaN,NaN


In [3]:
# create empty dataframe with the same columns as `data`
summary = pd.DataFrame(columns=data.columns.to_list())
summary.loc['count'] = data.count()
mode = data.mode().head(1)
mode.reset_index(drop=True, inplace=True)
mode.rename(index={0: 'top'}, inplace=True)
summary = pd.concat([summary, mode])
freq = data.apply(lambda x: x.value_counts(dropna=True).max())
summary.loc['freq'] = freq

unique = data.nunique(dropna=True)
summary.loc['unique'] = unique

summary

,last_updated,Create Date,Do you feel the True North report identified the key challenges and opportunities facing the region?,Are you interested in attending future True North events?,Are you currently a B Corp or in the process of becoming a B Corp?,Would you be interested in hearing more from Brabners about the B Corp process?,Company Name,City,Industry,sector,location,Which theme of the True North report do you most identify with and could support activity around?,How would you like to be involved with the True North network?,company_size
count,275,275,199,199,198,197,270,225,193,199,71,200,66,71
top,2024-05-15 18:59:00,2023-05-01 00:00:00,Yes,Yes,No,No,Audacia,London,Professional Training & Coaching,Education,Greater Manchester,"People, skills and the future",Keeping informed about the latest True North n...,0-9
freq,13,61,192,197,171,106,2,33,23,9,12,60,23,32
unique,235,194,2,2,2,2,254,95,54,155,26,14,7,5


In [4]:
# summary = data.describe()
summary.loc['top_percent_of_count'] = (summary.loc['freq'] * 100 / summary.loc['count']).astype(float).round(1)

In [5]:
summary

,last_updated,Create Date,Do you feel the True North report identified the key challenges and opportunities facing the region?,Are you interested in attending future True North events?,Are you currently a B Corp or in the process of becoming a B Corp?,Would you be interested in hearing more from Brabners about the B Corp process?,Company Name,City,Industry,sector,location,Which theme of the True North report do you most identify with and could support activity around?,How would you like to be involved with the True North network?,company_size
count,275,275,199,199,198,197,270,225,193,199,71,200,66,71
top,2024-05-15 18:59:00,2023-05-01 00:00:00,Yes,Yes,No,No,Audacia,London,Professional Training & Coaching,Education,Greater Manchester,"People, skills and the future",Keeping informed about the latest True North n...,0-9
freq,13,61,192,197,171,106,2,33,23,9,12,60,23,32
unique,235,194,2,2,2,2,254,95,54,155,26,14,7,5
top_percent_of_count,4.7,22.2,96.5,99.0,86.4,53.8,0.7,14.7,11.9,4.5,16.9,30.0,34.8,45.1


In [6]:
company_names = pd.Series(data['Company Name'].sort_values())
print(company_names.unique())

['2030hub' 'ACAS' 'AHR Architects' 'Abeceder ltd'
 'Access Creative College' 'Accountable Recruitment Limited'
 'Active Profile' 'Advanced Manufacturing Research Centre' 'Agent'
 'Agile Automations' 'Ahead Partnership' 'Andrea Nixon Consulting'
 'Anticus Partners Limited' 'Arcadis' 'Armstrong Watson'
 'Atkin Jones Limited' 'Audacia' 'Avison Young' 'Azets'
 'B engineering group' 'BHP LLP' 'BOW' 'BPP' 'BRUNTWOOD' 'BWB'
 'Bank of England' 'Barton Legal Limited' 'Big Hand 4 Business'
 'Blackpool Council' 'Boo Coaching & Consulting'
 'Booking.com (UK HQ in Manchester)'
 'Bradford District Place Making & Investment Partnership and other non-exec roles'
 'Brompton Bicycle Ltd' 'Business Doctors' 'Business Health Institute'
 'Business of Science Ltd' 'CBI' 'CH Marketing & Property'
 'Cabron Happy World' 'Careers and Enterprise Company' 'Cazenove Capital'
 'Centre for Global Eco-Innovation' 'Chris Shakespeare Coaching'
 'Civic Engineers' 'Codurance Ltd' 'Cortus Advisory Group Limited'
 'DES Ene

There is LLoyds banking group and lloyds bank plc - we're assuming these are different companies.

In [7]:
# convert the month column to a datetime object
data['month'] = pd.to_datetime(data['Create Date'])

#convert the item to a formatted value in yyyy-mm format.
data['month_formatted'] = data['month'].apply(datetime.strftime, format='%Y-%m').sort_index(ascending=True)

In [8]:
# take the number of members and calculate the number that joined each month.
monthly_members = pd.DataFrame(data['month_formatted'].value_counts(ascending=False)).reset_index()
monthly_members['start_of_month'] = monthly_members['month_formatted'].astype(str) + '-01'

In [9]:
monthly_orgs = data.drop_duplicates(subset='Company Name', keep='last')
monthly_orgs = pd.DataFrame(monthly_orgs['month_formatted'].value_counts(ascending=False)).reset_index()
monthly_orgs['start_of_month'] = monthly_orgs['month_formatted'].astype(str) + '-01'

In [10]:
def decimal_date(data):
    # make a unix timestamp column
    data['timestamp'] = pd.to_datetime(data['month_formatted'], format='%Y-%m').astype(int) / 10**9
    # make a decimal date and round to 2dp.
    data['year'] = data['timestamp'].div((86400*365.25)).add(1970).round(2)
    # drop the timestamp column
    data.drop(columns='timestamp', inplace=True) 
    # set year and formatted month as the index so they aren't included in the cumsum.
    data.set_index(['year', 'month_formatted', 'start_of_month'], inplace=True, append=True)

    return data

In [11]:
def calculate_cumsum(data, count_name):
    # order by date, then do the cumsum. reset the index, drop the original index column as not needed
    data = pd.DataFrame(data.sort_index(level=2).cumsum(skipna=True).reset_index().drop(columns='level_0'))
    data.rename(columns={'count': f'{count_name}'}, inplace=True)
    return data

In [12]:
# apply above functions to data
cs_monthly_members = calculate_cumsum(decimal_date(monthly_members), count_name='individuals')
cs_monthly_orgs = calculate_cumsum(decimal_date(monthly_orgs), count_name='orgs')

In [13]:
# merged the two dataframes
cs_merged = cs_monthly_members.merge(cs_monthly_orgs, how='inner', on=['year', 'month_formatted', 'start_of_month'])

In [14]:
# write to file
cs_merged.to_csv(os.path.join(SRC_DIR,'overview/membership/_data/cumsum.csv'), index=False)

Make a bar chart for locations.

In [15]:
locations = data.copy()
split_locs = locations['location'].str.split(';').apply(pd.Series, 1).stack()
split_locs.index = split_locs.index.droplevel(-1)
split_locs.name = 'location'
del locations['location']
locations = locations.join(split_locs)
locations['location'] = locations['location'].str.lstrip()
top_locations = locations['location'].value_counts().reset_index().head(5)
top_locations.to_csv(os.path.join(SRC_DIR, 'overview/membership/_data/top_locations.csv'), index=False)

/var/folders/5k/p_091y6n6rscp__0p77_ynv80000gn/T/ipykernel_7029/694549953.py:2: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  split_locs = locations['location'].str.split(';').apply(pd.Series, 1).stack()


Calculating some summary stats

In [16]:
total_members = len(data.index)

total_companies = summary.loc['unique', 'Company Name']

top_company_size = summary.loc['top', 'company_size']

top_company_size_pct = summary.loc['top_percent_of_count', 'company_size']

top_industry = summary.loc['top', 'Industry']

top_industry_pct = summary.loc['top_percent_of_count', 'Industry']

In [17]:
data['City']

0          Garstang
1        Altrincham
2               NaN
3            London
4             Leeds
           ...     
270             NaN
271          London
272             NaN
273    Saint Helens
274         Preston
Name: City, Length: 275, dtype: object

### Word frequency in the sector column

In [18]:
def normalize_string(s):
    # Convert to lowercase and remove non-alphanumeric characters (keeping spaces)
    try:
        s = s.lower()
    except:
        print(f"'{s}' is not a stirng type. Converting to string\n")
        return str(s)
    s = re.sub(r'[^a-z0-9\s]', '', s)
    return s
data['normalized_sector'] = data['sector'].apply(normalize_string).str.split(';')
 
# Flatten the list and further split by spaces to handle multi-word strings
all_words = [word for sublist in data['normalized_sector'] for item in sublist for word in item.split()]
 
# Count the occurrences of each word
word_counts = Counter(all_words)
 
# Find the most common word
most_common_words = word_counts.most_common(10)
least_common_words = word_counts.most_common()

banned_words = ['and', 'or', 'of', 'it', 'the', 'for', 'with', 'we', 'nan']
# print("The 10 most common words, not including 'and' are:") 
# for word, count in most_common_words:
#     if word == banned_words:
#         continue
#     print(f"'{word}' with {count} occurences")

print("The most common words, with at least 2 occurences are:")
words = []
counts = []
for word, count in least_common_words:
    if word in banned_words:
        continue
    if word == 'estate':
        continue
    if count >= 10:
        if word == 'real':
            words.append('real estate')
        else:
            words.append(word)
        counts.append(count)
        print(f"'{word}' with {count} occurences")

sector_counts = pd.DataFrame(data={'name': words, 'count': counts}).set_index('name')
# sector_counts['colour'] = round((sector_counts['count'] - min(sector_counts['count'])) / sector_counts['count'].max(), 3)
sector_counts.to_csv(os.path.join(SRC_DIR, 'overview/membership/_data/sector_word_counts.csv'))
# sector_strings = [item for sublist in data['sector'].str.split(';') for item in sublist]
# string_counts = Counter(sector_strings)
# print(string_counts.most_common())


'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string



In [19]:
advisory_council = 11

if summary.loc['top', "Are you currently a B Corp or in the process of becoming a B Corp?"] == 'No':
    b_corps_pct = round(100 - summary.loc['top_percent_of_count', "Are you currently a B Corp or in the process of becoming a B Corp?"], 1)
else:
    summary.loc['top_percent_of_count', "Are you currently a B Corp or in the process of becoming a B Corp?"]

membership_increase = cs_merged['individuals'].pct_change().mul(100).iloc[-1].round(1)

northern_stars = len(pd.read_csv(os.path.join(SRC_DIR, 'overview/northern-stars/_data/northern_stars.csv')))

names = ["Total members", 
         "Total companies", 
         "Membership increase", 
         f"Companies with {top_company_size} employees", 
         "Top industry", 
         "Geographic reach", 
         "Advisory council", 
         "Northern stars", 
         "B Corps"]

values = [total_members, 
          total_companies, 
          membership_increase, 
          top_company_size_pct, 
          top_industry_pct, 
          "4", 
          advisory_council, 
          northern_stars, 
          b_corps_pct]

footnotes = ["People", 
             "Unique companies", 
             "Since last month", 
             "Of members work in companies of this size", 
             f"Of members work in {top_industry}", 
             "placeholder", 
             "Members represent the network on the True North advisory council", 
             "Companies have been featured as Northern Stars", 
             "Of member's organisations are B Corps or are joining"]

posts = ['','','%','%','%','','','','%']

urls = ['/overview/membership', '/overview/membership', '/overview/membership', '', '', '', 'https://www.brabners.com/insights/true-north/true-north-advisory-council-launches', '/overview/northern-stars', '/themes/sustainable-growth/b-corporations']
dashboard = pd.DataFrame(data={'name':names, 'value': values, 'footnote': footnotes, 'post': posts, 'url': urls})

dashboard

,name,value,footnote,post,url
0,Total members,275,People,,/overview/membership
1,Total companies,254,Unique companies,,/overview/membership
2,Membership increase,1.1,Since last month,%,/overview/membership
3,Companies with 0-9 employees,45.1,Of members work in companies of this size,%,
4,Top industry,11.9,Of members work in Professional Training & Coa...,%,
5,Geographic reach,4,placeholder,,
6,Advisory council,11,Members represent the network on the True Nort...,,https://www.brabners.com/insights/true-north/t...
7,Northern stars,5,Companies have been featured as Northern Stars,,/overview/northern-stars
8,B Corps,13.6,Of member's organisations are B Corps or are j...,%,/themes/sustainable-growth/b-corporations


In [20]:
dashboard.to_csv(os.path.join(SRC_DIR, 'overview/membership/_data/true_north_members_list.csv'), index=False)

In [21]:
time_updated('src/overview/membership/_data/updated.yaml')

Timestamp added to file src/overview/membership/_data/updated.yaml
